# CURVE_FIT

## Overview
The `CURVE_FIT` function fits a user-defined model function to data using non-linear least squares, leveraging SciPy's [`curve_fit` method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). It is ideal for regression, parameter estimation, and curve fitting directly in Excel.  If you want to fit several models to the same data this can be faster than setting up the Excel Solver for each model.  This is a simplified implementation, you can extend it to support additional features like bounds, constraints, and more complex models.

Non-linear least squares fitting seeks to find parameters $\theta$ that minimize the sum of squared residuals:

```math
S(\theta) = \sum_{i=1}^n (y_i - f(x_i, \theta))^2
```

where $f(x_i, \theta)$ is the model function, $x_i$ are the input data, and $y_i$ are the observed values. SciPy's `curve_fit` uses algorithms such as Trust Region Reflective (TRF), Dogleg (dogbox), and Levenberg-Marquardt (LM) to solve this optimization problem.

This example function is provided as-is without any representation or warranty of accuracy.

## Usage
To use the function in Excel:

```excel
=CURVE_FIT(model, xdata, ydata, [p_zero])
```
  - `model` (string, required): Model function as a string, e.g., "a * x + b".
  - `xdata` (2D list, required): Input x values.
  - `ydata` (2D list, required): Observed y values.
  - `p_zero` (2D list, optional): Initial parameter guesses.

The function returns the fitted parameter values as a single row 2D list, or an error message string if the fit fails.

## Examples

### Fitting a Straight Line (y = a * x + b)

**Sample Input Data:**

| x | y |
|---|---|
| 1 | 2 |
| 2 | 4 |
| 3 | 6 |

```excel
=CURVE_FIT("a * x + b", {1;2;3}, {2;4;6}, {1,1})
```
**Sample Output:**

| a   | b   |
|-----|-----|
| 2.0 | 0.0 |

### Fitting an Exponential Model (y = a * exp(b * x))

**Sample Input Data:**

| x | y  |
|---|----|
| 1 | 2.7|
| 2 | 7.4|
| 3 | 20.1|

```excel
=CURVE_FIT("a * exp(b * x)", {1;2;3}, {2.7;7.4;20.1}, {1,1})
```
**Sample Output:**

| a   | b   |
|-----|-----|
| 1.0 | 1.0 |

### Fitting a Decaying Exponential Model (y = a * exp(-b * x) + c)

**Sample Input Data:**

| x | y  |
|---|----|
| 0 | 3.4|
| 1 | 2.7|
| 2 | 1.6|
| 3 | 1.1|
| 4 | 0.7|
| 5 | 0.6|

```excel
=CURVE_FIT("a * exp(-b * x) + c", {0;1;2;3;4;5}, {3.4;2.7;1.6;1.1;0.7;0.6}, {2,1,0.5})
```
**Sample Output:**

| a   | b   | c   |
|-----|-----|-----|
| 2.0 | 1.0 | 0.5 |

## Limitations
- The model string must use `x` as the independent variable and parameter names (e.g., `a`, `b`).
- The number of initial guesses (if provided) must match the number of parameters in the model.
- If the fit fails, an error message is returned as a string.
- Only methods supported by SciPy's `curve_fit` are allowed (`trf`, `dogbox`, `lm`).

In [ ]:
import numpy as np
from scipy.optimize import curve_fit as scipy_curve_fit
import math
SAFE_GLOBALS = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
SAFE_GLOBALS["np"] = np
SAFE_GLOBALS["numpy"] = np
SAFE_GLOBALS["exp"] = np.exp
SAFE_GLOBALS["log"] = np.log
SAFE_GLOBALS["sin"] = np.sin
SAFE_GLOBALS["cos"] = np.cos
SAFE_GLOBALS["tan"] = np.tan
SAFE_GLOBALS["abs"] = abs
SAFE_GLOBALS["pow"] = pow

def curve_fit(model, xdata, ydata, p_zero=None):
    """
    Fits a model to data using scipy.optimize.curve_fit.

    Args:
        model (str): Model function as a string, e.g., "a * x + b"
        xdata (list[list[float]]): 2D list of x values
        ydata (list[list[float]]): 2D list of y values
        p_zero (list[list[float]], optional): 2D list of initial parameter guesses

    Returns:
        list[list[float]]: Fitted parameter values as a single row, or error message string

    This example function is provided as-is without any representation or warranty of accuracy.
    """
    try:
        x = np.array(xdata).flatten()
        y = np.array(ydata).flatten()
        import re
        param_names = re.findall(r'\b[a-zA-Z_]\w*\b', model)
        param_names = [name for name in param_names if name not in ("x", "exp", "log", "sin", "cos", "tan", "abs", "pow")]
        param_names = list(dict.fromkeys(param_names))
        n_params = len(param_names)
        if p_zero is not None:
            p_zero_arr = np.array(p_zero).flatten()
            if len(p_zero_arr) != n_params:
                return f"Number of initial guesses (p_zero) does not match number of parameters in model: {param_names}"
        else:
            p_zero_arr = None
        def model_func(x, *params):
            local_dict = dict(zip(param_names, params))
            local_dict["x"] = x
            try:
                return eval(model, SAFE_GLOBALS, local_dict)
            except Exception as e:
                return f"Model evaluation error: {e}"
        popt, _ = scipy_curve_fit(model_func, x, y, p0=p_zero_arr, maxfev=10000)
        # Round fitted parameters to 0.01
        popt_rounded = np.round(popt, 2)
        return [popt_rounded.tolist()]
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    ["a * x + b", [[1], [2], [3]], [[2.1], [3.8], [6.2]], [[1, 1]]],
    ["a * exp(b * x)", [[1], [2], [3]], [[2.5], [7.8], [19.5]], [[1, 1]]],
    ["a * exp(-b * x) + c", [[0], [1], [2], [3], [4], [5]], [[3.4], [2.7], [1.6], [1.1], [0.7], [0.6]], [[2, 1, 0.5]]]
]

def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row) for row in val)
    return False

import pytest
@pytest.mark.parametrize("model, xdata, ydata, p_zero", demo_cases)
def test_demo_cases(model, xdata, ydata, p_zero):
    result = curve_fit(model, xdata, ydata, p_zero)
    print(f"test_demo_cases output for {model}: {result}")
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

def test_invalid_param_count():
    result = curve_fit("a * x + b", [[1], [2], [3]], [[2], [4], [6]], [[1]])
    print(f"test_invalid_param_count output: {result}")
    assert isinstance(result, str) and "does not match" in result

ipytest.run('-s')

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import io
import base64

def curve_fit_with_plot(model, xdata, ydata, p_zero=None):
    result = curve_fit(model, xdata, ydata, p_zero)
    # Prepare plot
    fig, ax = plt.subplots()
    x = np.array(xdata).flatten()
    y = np.array(ydata).flatten()
    ax.scatter(x, y, label="Data", color="blue")
    if isinstance(result, list) and isinstance(result[0], list):
        # Try to plot fitted curve
        try:
            import re
            param_names = re.findall(r'\b[a-zA-Z_]\w*\b', model)
            param_names = [name for name in param_names if name not in ("x", "exp", "log", "sin", "cos", "tan", "abs", "pow")]
            param_names = list(dict.fromkeys(param_names))
            params = result[0]
            x_fit = np.linspace(np.min(x), np.max(x), 100)
            local_dict = dict(zip(param_names, params))
            local_dict["x"] = x_fit
            y_fit = eval(model, SAFE_GLOBALS, local_dict)
            ax.plot(x_fit, y_fit, label="Fitted", color="red")
            ax.legend()
        except Exception as e:
            ax.text(0.5, 0.5, f"Plot error: {e}", ha='center')
    else:
        ax.text(0.5, 0.5, "Fit failed", ha='center')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")
    html = f'<img src="data:image/png;base64,{img_base64}" style="max-width:100%;">'
    return result, html

demo = gr.Interface(
    fn=curve_fit_with_plot,
    inputs=[
        gr.Textbox(label="Model (Python expression, e.g. a * x + b)", value=demo_cases[0][0]),
        gr.Dataframe(label="x data", headers=["x"], type="array", row_count=3, col_count=1, value=demo_cases[0][1]),
        gr.Dataframe(label="y data", headers=["y"], type="array", row_count=3, col_count=1, value=demo_cases[0][2]),
        gr.Dataframe(label="Initial parameter guesses (row, optional)", headers=["a", "b", "c"], type="array", row_count=1, col_count=3, value=demo_cases[0][3])
    ],
    outputs=[
        gr.Dataframe(label="Fitted Parameters (row)", headers=["a", "b", "c"], type="array"),
        gr.HTML(label="Fit Plot")
    ],
    examples=demo_cases,
    description="Fit a mathematical model to your data using non-linear least squares. Enter your model as a Python expression (e.g., `a * x + b`), provide your data, and set initial guesses for the parameters (optional). Use the demo examples below to see typical use cases.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()